In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Data loading
df_23_24_sales = pd.read_excel('SECONDARY YTD 23-24 Fragrance - Till February.xlsx', sheet_name="BASE DATA")
df_22_23_sales = pd.read_excel('Sales Data - Pivot (FY 16-23).xlsx', sheet_name="FY 22-23")

In [3]:
#Targetting the data for the LFS Channel
df_22_23_sales_lfs = df_22_23_sales[df_22_23_sales['Channel-1'] == 'LFS']
df_23_24_sales_lfs = df_23_24_sales[df_23_24_sales['Channel-2'] == 'LFS']

In [4]:
df_22_23_sales_lfs.rename(columns={'Channel-1': 'Channel_lfs'}, inplace=True)
df_23_24_sales_lfs.rename(columns={'Channel-2': 'Channel_lfs'}, inplace=True)
df_22_23_sales_lfs.rename(columns={'Variant Name': 'Varient_Name'}, inplace=True)
df_23_24_sales_lfs.rename(columns={'Varient Name2': 'Varient_Name'}, inplace=True)

In [5]:
#Combination of both the data frames 
combined_df_lfs = pd.concat([df_22_23_sales_lfs, df_23_24_sales_lfs], ignore_index=True)
combined_df_lfs.rename(columns={' Net UCP': 'Net UCP'}, inplace=True)
column_names = combined_df_lfs.columns.tolist()
print(column_names)

['Year', 'Month', 'Month Key', 'QTR', 'Region', 'Invoice Date', 'Material', 'Quantity', 'Gender', 'Brand', 'Channel', 'Franchisee store', 'Bill to Party code', 'Bill to Party Name', 'MRP', 'Gross UCP', 'Net UCP', 'Bill to Party City', 'ShiptopartyStateCode', 'Billing grp', 'Channel_lfs', 'Varient_Name', 'Collection']


In [6]:
combined_df_lfs['Gender'].value_counts()

Gender
G    207062
L    154384
P     16987
U       635
Name: count, dtype: int64

In [7]:
combined_df_lfs['Brand'].value_counts()

Brand
TF    375396
Tf      2520
FP      1152
Name: count, dtype: int64

In [8]:
#Replace the data of the 'Tf' from the 'TF' respectively
demo_brand_check = combined_df_lfs[combined_df_lfs['Brand'] == 'Tf']
combined_df_lfs['Brand'] = combined_df_lfs['Brand'].replace('Tf', 'TF')
combined_df_lfs['Brand'].value_counts()

Brand
TF    377916
FP      1152
Name: count, dtype: int64

In [9]:
combined_df_lfs.head(1)

,Year,Month,Month Key,QTR,Region,Invoice Date,Material,Quantity,Gender,Brand,...,Bill to Party Name,MRP,Gross UCP,Net UCP,Bill to Party City,ShiptopartyStateCode,Billing grp,Channel_lfs,Varient_Name,Collection
0,2022-23,2022-04-01,Apr,Q1,South,2022-04-01 00:00:00,NFM03DQ1,4.0,G,TF,...,CT-BENGALURU-RESIDENCYROAD,499,1996.0,1996.0,Bangalore,KARNATAKA,LFS,LFS,3. 150 M Amalfi Men deo,g. Premium Deo


In [10]:
mapping = {
    'd. Gift Pack': 'Gift Pack',
    'g. Premium Deo': 'Premium Deo',
    'a. Classic 100 ml': 'Classic 100 ml',
    'f. Classic 20 ml': 'Classic 20 ml',
    'b. Classic 50 ml': 'Classic 50 ml',
    'e. Aqua': 'Aqua',
    'g. Escapade Deo': 'Escapade Deo',
    'c. Escapade': 'Escapade',
    'j1. Nox 100 ml': 'Nox 100 ml',
    'I. Tales 100ml': 'Tales 100ml',
    'j2. Noura W 100ml': 'Noura W 100ml',
    'h. Fastrack': 'Fastrack',
    'h. Fastrack-Bold': 'Fastrack-Bold',
    'x. Bohemian': 'Bohemian',
    ' Premium Deo':'Premium Deo',
    ' Escapade Deo':'Escapade Deo',
    ' Bohemian':'Bohemian',
    ' Deo Bundle Pack':'Deo Bundle Pack'
}
combined_df_lfs['Collection'] = combined_df_lfs['Collection'].replace(mapping)
combined_df_lfs['Collection'].value_counts()

Collection
Gift Pack          62820
Classic 100 ml     61611
Premium Deo        58719
Classic 20 ml      52374
Aqua               29753
Classic 50 ml      26505
Escapade Deo       24448
Escapade           19996
Nox 100 ml         12883
Tales 100ml         8704
Noura W 100ml       7139
Deo Bundle Pack     6384
Nox 20 ml           2661
Noura 20 ml         2586
Classic Flanker     1270
Fastrack            1190
Skinn Mist            15
Bohemian               6
Fastrack-Bold          4
Name: count, dtype: int64

In [11]:
from ydata_profiling import ProfileReport
prof = ProfileReport(combined_df_lfs)
prof.to_file(output_file='ARIMA.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
demo_region_check = combined_df_lfs[combined_df_lfs['Region'] == 'NORTH']
combined_df_lfs['Region'] = combined_df_lfs['Region'].replace('NORTH', 'North')
combined_df_lfs['Region'].value_counts()

Region
West     105498
North    100417
South     90644
East      68152
Name: count, dtype: int64

In [13]:
demo_collection_check=combined_df_lfs.copy()
demo_collection_check['min_dt'] = demo_collection_check.groupby('Collection')['Invoice Date'].transform('min')
demo_collection_check['max_dt'] = demo_collection_check.groupby('Collection')['Invoice Date'].transform('max')
demo_collection_check = demo_collection_check[['Collection','min_dt','max_dt']].drop_duplicates().reset_index(drop=True)

demo_collection_check['tag'] = demo_collection_check.apply(lambda x: 1 if ((x['min_dt'].year == demo_collection_check['min_dt'].min().year) & (x['min_dt'].month == demo_collection_check['min_dt'].min().month)) & ((x['max_dt'].year == demo_collection_check['max_dt'].max().year) & (x['max_dt'].month == demo_collection_check['max_dt'].max().month)) else 0,axis=1)

demo_collection_check

,Collection,min_dt,max_dt,tag
0,Premium Deo,2022-04-01,2024-02-29,1
1,Classic 20 ml,2022-04-01,2024-02-29,1
2,Escapade Deo,2022-04-01,2024-02-29,1
3,Tales 100ml,2022-04-01,2024-02-29,1
4,Classic 100 ml,2022-04-01,2024-02-29,1
5,Gift Pack,2022-04-01,2024-02-29,1
6,Classic 50 ml,2022-04-01,2024-02-29,1
7,Escapade,2022-04-01,2024-02-29,1
8,Aqua,2022-04-01,2024-02-29,1
9,Fastrack,2022-04-01,2024-02-27,1


In [14]:
demo_data = combined_df_lfs.copy()

In [15]:
#Getting the data for the continued collection
demo_collection_check_continued = demo_collection_check.loc[demo_collection_check['tag']==1,'Collection']
# demo_collection_check_continued
demo_data = demo_data[demo_data['Collection'].isin(demo_collection_check_continued)]
demo_data.shape
#Filtered data for the continued fraganances

(359003, 23)

In [16]:
#Now we are going to analyze the data Month wise for the collections of 'demo_collection_check_continued'
demo_data_monthly = demo_data.groupby(['Year','QTR','Month Key','Collection','Region'],as_index=False).agg({'Net UCP':'sum','Quantity':'sum'})
demo_data_monthly.tail(10)

,Year,QTR,Month Key,Collection,Region,Net UCP,Quantity
913,2023-24,Q4,Jan,Classic 50 ml,South,72389.0,41.0
914,2023-24,Q4,Jan,Escapade,South,42042.0,16.0
915,2023-24,Q4,Jan,Escapade Deo,North,499.0,1.0
916,2023-24,Q4,Jan,Escapade Deo,South,6487.0,13.0
917,2023-24,Q4,Jan,Gift Pack,North,21349.5,11.0
918,2023-24,Q4,Jan,Gift Pack,South,90004.0,41.0
919,2023-24,Q4,Jan,Nox 100 ml,North,3995.0,1.0
920,2023-24,Q4,Jan,Nox 100 ml,South,27965.0,7.0
921,2023-24,Q4,Jan,Premium Deo,North,998.0,2.0
922,2023-24,Q4,Jan,Premium Deo,South,24451.0,49.0


In [25]:
#Now we are going to draw the histplot for the 'Net UCP' and 'Quantity' from the 'demo_data_monthly' for the 'Collection; 'Classic 100 ml'
demo_data_monthly_classic_100 = demo_data_monthly[demo_data_monthly['Collection']=='Classic 100 ml']
demo_data_monthly_classic_100.head(10)
sns.histplot(data=demo_data_monthly_classic_100, x='Net UCP', bins=1000)

<Axes: title={'center': 'MaxAbs Scaled Quantity Over Time'}, xlabel='Month', ylabel='MaxAbs Scaled Quantity'>

In [18]:
from scipy.stats import skew
columns_to_check = ['Net UCP', 'Quantity']
for col in columns_to_check:
    print(f"Column: {col}")
    print(f"Skewness: {skew(demo_data_monthly_classic_100[col])}")
    plt.figure(figsize=(8, 5))
    sns.histplot(demo_data_monthly_classic_100[col], kde=True)
    plt.show()

Column: Net UCP
Skewness: 3.233555383558647
Column: Quantity
Skewness: 3.0882987319795863


In [19]:
#Now we are going to draw the histplot for the 'Net UCP' and 'Quantity' from the 'demo_data_monthly' for all the 'Collection'
sns.histplot(data=demo_data_monthly, x='Net UCP', bins=1000)

<Axes: xlabel='Quantity', ylabel='Count'>

In [20]:
from scipy.stats import skew
columns_to_check = ['Net UCP', 'Quantity']
for col in columns_to_check:
    print(f"Column: {col}")
    print(f"Skewness: {skew(demo_data_monthly[col])}")
    plt.figure(figsize=(8, 5))
    sns.histplot(demo_data_monthly[col], kde=True)
    plt.show()

Column: Net UCP
Skewness: 2.6996419853421716
Column: Quantity
Skewness: 2.2264795139522557


In [21]:
#Now we are going to do Feature scaling using the Standardization (Z-Score Normalization) for the 'Net UCP' and 'Quantity' from the 'demo_data_monthly' for all the 'Collection'
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = demo_data_monthly[['Net UCP', 'Quantity']]
scaled_data = scaler.fit_transform(scaled_data)
scaled_data = pd.DataFrame(scaled_data, columns=['Net UCP', 'Quantity'])
demo_data_monthly['Z_Score_Net_UCP'] = scaled_data['Net UCP']
demo_data_monthly['Z_Score_Quantity'] = scaled_data['Quantity']

# Plotting the Z-scores
plt.figure(figsize=(12, 6))

# Plot for Z_Score_Net_UCP
plt.subplot(2, 1, 1)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['Z_Score_Net_UCP'], marker='o', linestyle='-', color='b')
plt.title('Z-Score of Net UCP Over Time')
plt.xlabel('Month')
plt.ylabel('Z-Score of Net UCP')

# Plot for Z_Score_Quantity
plt.subplot(2, 1, 2)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['Z_Score_Quantity'], marker='o', linestyle='-', color='r')
plt.title('Z-Score of Quantity Over Time')
plt.xlabel('Month')
plt.ylabel('Z-Score of Quantity')

plt.tight_layout()
plt.show()

In [22]:
#Now we are going to do Feature scaling using the Min-Max Scaling (Normalization) for the 'Net UCP' and 'Quantity' from the 'demo_data_monthly' for all the 'Collection'
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = demo_data_monthly[['Net UCP', 'Quantity']]
scaled_data = scaler.fit_transform(scaled_data)
scaled_data = pd.DataFrame(scaled_data, columns=['Net UCP', 'Quantity'])
demo_data_monthly['MinMax_Net_UCP'] = scaled_data['Net UCP']
demo_data_monthly['MinMax_Quantity'] = scaled_data['Quantity']

# Plotting the Min-Max Scaled values
plt.figure(figsize=(12, 6))

# Plot for MinMax_Net_UCP
plt.subplot(2, 1, 1)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['MinMax_Net_UCP'], marker='o', linestyle='-', color='b')
plt.title('MinMax Scaled Net UCP Over Time')
plt.xlabel('Month')
plt.ylabel('MinMax Scaled Net UCP')

# Plot for MinMax_Quantity
plt.subplot(2, 1, 2)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['MinMax_Quantity'], marker='o', linestyle='-', color='r')
plt.title('MinMax Scaled Quantity Over Time')
plt.xlabel('Month')
plt.ylabel('MinMax Scaled Quantity')

plt.tight_layout()
plt.show()

In [23]:
#Now we are going to do Feature scaling using the Robust Scaling for the 'Net UCP' and 'Quantity' from the 'demo_data_monthly' for all the 'Collection'
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaled_data = demo_data_monthly[['Net UCP', 'Quantity']]
scaled_data = scaler.fit_transform(scaled_data)
scaled_data = pd.DataFrame(scaled_data, columns=['Net UCP', 'Quantity'])
demo_data_monthly['Robust_Net_UCP'] = scaled_data['Net UCP']
demo_data_monthly['Robust_Quantity'] = scaled_data['Quantity']

# Plotting the Robust Scaled values
plt.figure(figsize=(12, 6))

# Plot for Robust_Net_UCP
plt.subplot(2, 1, 1)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['Robust_Net_UCP'], marker='o', linestyle='-', color='b')
plt.title('Robust Scaled Net UCP Over Time')
plt.xlabel('Month')
plt.ylabel('Robust Scaled Net UCP')

# Plot for Robust_Quantity
plt.subplot(2, 1, 2)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['Robust_Quantity'], marker='o', linestyle='-', color='r')
plt.title('Robust Scaled Quantity Over Time')
plt.xlabel('Month')
plt.ylabel('Robust Scaled Quantity')

plt.tight_layout()
plt.show()



In [24]:
#Now we are going to do Feature scaling using the Absolute Maximum Scaling for the 'Net UCP' and 'Quantity' from the 'demo_data_monthly' for all the 'Collection'
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
scaled_data = demo_data_monthly[['Net UCP', 'Quantity']]
scaled_data = scaler.fit_transform(scaled_data)
scaled_data = pd.DataFrame(scaled_data, columns=['Net UCP', 'Quantity'])
demo_data_monthly['MaxAbs_Net_UCP'] = scaled_data['Net UCP']
demo_data_monthly['MaxAbs_Quantity'] = scaled_data['Quantity']

# Plotting the MaxAbs Scaled values
plt.figure(figsize=(12, 6))

# Plot for MaxAbs_Net_UCP
plt.subplot(2, 1, 1)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['MaxAbs_Net_UCP'], marker='o', linestyle='-', color='b')
plt.title('MaxAbs Scaled Net UCP Over Time')
plt.xlabel('Month')
plt.ylabel('MaxAbs Scaled Net UCP')

# Plot for MaxAbs_Quantity
plt.subplot(2, 1, 2)
plt.plot(demo_data_monthly['Month Key'], demo_data_monthly['MaxAbs_Quantity'], marker='o', linestyle='-', color='r')
plt.title('MaxAbs Scaled Quantity Over Time')
plt.xlabel('Month')
plt.ylabel('MaxAbs Scaled Quantity')

plt.tight_layout()
plt.show()